In [ ]:
import os
from groq import Groq
import pandas as pd
import numpy as np

In [ ]:
apiKey = "fill_it_here"

In [ ]:
client = Groq(
    # api_key=os.environ.get("GROQ_API_KEY"),
    api_key= apiKey,
)

In [ ]:
dataset = pd.read_csv("keyphrases/done/keyphrases_AmazonMixtral_Done.csv")

In [ ]:
dataset

In [ ]:
score_counts_balanced = dataset['sentiment'].value_counts().sort_index()
print(score_counts_balanced)

In [ ]:
train_data = dataset[['content', 'sentiment', 'Merged keyphrases_reduced 2']]
train_data['row_number'] = train_data.index

In [ ]:
train_data

In [ ]:
list = []
i = 0
for index, row in train_data.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - very negative, 2 - negative, 3 - neutral, 4 - positive, 5 - very positive. Here are examples to guide your classification:

                1. Comment: "Not sure why Amazon thinks it's OK to hijack my long press menu to add their stupid search icon there with no option to remove it. Oh well, now I have one less app taking up space on my phone."
                Topics / Keyphrases: ['social media', 'digital marketing', 'email marketing', 'content marketing', 'social media marketing', 'search engine optimization', 'influencer marketing', 'mobile marketing', 'video marketing', 'email campaigns']
                Label: 1

                2. Comment: "App is now incredibly laggy to the point I'm unable to scroll down to read reviews or look at pictures. Makes it impossible to shop. 0/5 stars. Can't even use the app."
                Topics / Keyphrases: ['Amazon App problems', 'User experience decline', 'App stability issues', 'Recent update difficulties', '5 seconds load time', 'Negative feedback', 'Functionality errors', 'Requires immediate fix', 'User frustration', 'Potential loss of customer']
                Label: 1

                3. Comment: "The absurd experience I've had with Amazon over these past 2 weeks is nothing short of horrendous. I'm all for employing foreign people to deal with customer service but if they themselves are unable to properly handle serious issues what is the point. I have had to spend hours in my bank getting my card changed all so I can make my purchases and your agent has me remove and re-add it only for it to not be accepted then they transfer me 5 times and ask me the same stupid questions"
                Topics / Keyphrases: ['Amazon delivery issues', 'long delivery times', 'user frustrations', 'Prime membership concerns', 'app tracking problems', 'order checking difficulties', 'medical supply delivery', 'customer service communication', 'electric bicycle payment issues', 'inaccurate orders and refunds']
                Label: 1

                4. Comment: "terrible customer service, and they WILL send you broken and or defective products, or nothing at all! prime sucks, they treat you like trash weather you are a prime member who has spent thousands of dollars, or someone whos never spent a dime. seriously, go to the store yourself. amazon corps should be dismantled."
                Topics / Keyphrases: ['higher prices on Amazon', 'product display issues on Amazon', 'frustration with Amazon shopping experience', 'time-consuming product sorting', 'sponsored products concerns on Amazon', 'fake products on Amazon', 'need for seller verification', 'customer dissatisfaction with product display', 'poor delivery experience', 'terrible customer service']
                Label: 1

                5. Comment: "Cannot disable product videos. You search for something and right away videos start playing in the product pictures. Why you can't disable this. I want to focus on shipping not on idiotic videos! This feature is damn, unethical and borderline illegal. You need to be ashamed pushing this to your customers."
                Topics / Keyphrases: ['User experience issue with Amazon App', 'Inclusion of product videos in product pictures', 'Limitations in disabling product videos', 'Customer complaints on Google Play Store', 'Preference for product images over video demonstrations', 'Professional users' difficulty with video feature', 'Annoyance with TikTok-style influencers', 'Product touching element in videos', 'Ethical concerns over unrequested videos', 'Search results affected by unwanted videos']
                Label: 1

                6. Comment: "Too slow, i honestly don't know what happened after the new update it got too laggy, the app started crashing and it became a pain to shop from the app"
                Topics / Keyphrases: ['Amazon App issues', 'user experience decline', 'functionality affected', 'slow performance', 'app crashing', 'order tracking problem', 'money transfer delay', 'constant crashes', 'specific date bug', 'security concern']
                Label: 2

                7. Comment: "Latest app version is unreasonably buggy, especially with friend wish lists. They work fine directly from the website, but why have an app if we're constantly being forced back to the website due to flakes in the app. Google's inability (intent?) to play well with Kindle does make me wonder about the rest of the Amazon."
                Topics / Keyphrases: ['Amazon App performance issues', 'slow loading on multiple phones', 'hard to navigate UI', 'Google-Kindle conflict', 'forced back to website', 'Amazon reliability', 'minimal investment in app', 'connecting issues with Blizzard account', 'increasing prices and reducing service', 'user frustration']
                Label: 2

                8. Comment: "would love to Bulk my order to make it easier for Amazon, but when they can't get simple promised deliveries done, why should I try to help. if they tell you next day delivery it's going to take 2 or 3 days. like it or not. I don't care for that new way of over promising."
                Topics / Keyphrases: ['Amazon delivery issues', 'promised vs actual delivery times', 'user experience and Prime membership', 'prices increase with items', 'monopoly concerns and delayed deliveries', 'payment processing problems', 'damaged and stolen packages', 'product quality and refund processing', 'cost comparison with other vendors', 'alternatives to Amazon']
                Label: 2

                9. Comment: "The app for Android is incredibly inferior to the Apple app. I haven't been able to open my recent orders for months, it spins and then I get an error. I'm not sure why that functionality hasn't been fixed yet."
                Topics / Keyphrases: ['Amazon App issues', 'user experience decline', 'functionality affected', 'slow performance', 'app crashing', 'order tracking problem', 'money transfer delay', 'constant crashes', 'specific date bug', 'security concern']
                Label: 2

                10. Comment: "I'm tired of major app changes without warning. Perhaps user input should count for something. Can't stand having the search bar at the bottom now. Considering uninstalling and just using the website."
                Topics / Keyphrases: ['Amazon App changes', 'user experience', 'feedback on changes', 'Google Play Store comments', 'app design', 'lack of warning', 'user input', 'search bar location', 'app performance issues', 'user complaints']
                Label: 2

                11. Comment: "Please give an option to choose the new or the classic wish list interface. The new interface is completely unhelpful and only adds more clicks/taps to get to where I really want to be. Not to mention, for privacy reasons maybe one doesn't want *everything* appearing on the landing page."
                Topics / Keyphrases: ['Amazon App search bar location', 'User interface preference', 'Application features', 'User feedback', 'Instructional videos', 'Product links', 'Design change', 'User frustration', 'Wish list design', 'End user control']
                Label: 3

                12. Comment: "its okay. can you please bring back the ability to remove suggestions im not interested in. please allow me to either delete older purchases or make sure i stop being advertised stuff i absolutely do not want"
                Topics / Keyphrases: ['social media', 'digital marketing', 'email marketing', 'content marketing', 'social media marketing', 'search engine optimization', 'influencer marketing', 'mobile marketing', 'video marketing', 'email campaigns']
                Label: 3

                13. Comment: "It could be better but it is good. I think when you want to move some items from one folder to another I wish you could select more than one item at a time to move at once."
                Topics / Keyphrases: ['Amazon App user experience', 'app speed and performance', 'product prices and cost', 'shopping frequency and orders', 'delivery time and issues', 'specific delivery location', 'app interface and organization', 'shipping cost', 'dark mode and night time', 'multiple items selection and folders']
                Label: 3

                14. Comment: "Really like using this app as a disabled person it helps me get things that I can't get out to get. Lately though it's been crashing every time I go on, hopefully this is resolved soon."
                Topics / Keyphrases: ['Amazon App issues', 'Digital music errors', 'User frustrations', 'Inability to buy Kindle books', 'Lag when scrolling', 'Positive opinion vs. purchase issues', 'Problems with favorites list', 'Delivery experience issues', 'Searching for items challenges', 'App usability for disabled persons']
                Label: 3

                15. Comment: "The app is good and does the job. There is though no control on the fake reviews for products. The app also does not control any fake products. Unfortunately buyers trust the app and end up buying fake products."
                Topics / Keyphrases: ['Amazon App notifications', 'user preference', 'spam notifications', 'unwanted ads', 'package delivery notifications', 'user complaints', 'poorly rated products', 'app flaws', 'user interface update', 'buyer protection']
                Label: 3

                16. Comment: "So much choice. Great prices. Sometimes returns options are limited, but most sellers and Amazon are generally great at offering flexibility. Be wary of the default being to add a delivery charge for fast delivery and never ending offers of Prime!"
                Topics / Keyphrases: ['Amazon app purchases experience', 'Item quality and price concerns', 'Value for money decreasing', 'Smooth return process', 'Delivery expectations vs reality', 'Price discrepancy in app', 'Prime subscription value', 'Limited returns flexibility', 'International shipping cost', 'E-commerce packaging issues']
                Label: 4

                17. Comment: "Great place for wide range of products at the best prices. my only issue is the amount of time it takes to receive a refund. if not for the wait time would have been 5 starts. customer service not great but better than a lot of others that I have come across."
                Topics / Keyphrases: ['Amazon App experience', 'Higher prices and corporate profits', 'Employee benefits and pay', 'Healthcare and user preferences', 'Late deliveries and delay in transit', 'Warehouse processing and user satisfaction', 'Product delivery and return process', 'Checkout process and cost calculation', 'Wide range of products and Prime membership', 'Customer service and order discrepancies']
                Label: 4

                18. Comment: "Easy app, good product and pretty fast delivery for the most part."
                Topics / Keyphrases: ['Amazon App', 'user experience', 'product availability', 'positive review', 'online shopping', 'customer satisfaction', 'in-app customer reviews', 'rating accuracy', 'fast delivery', 'customer service']
                Label: 4

                19. Comment: "When everywhere else fails get it on Amazon 😂 Don't care for those 3rd party sellers when they tell you it was delivered & was not or we have to fight to get our products from them, they charge a higher price AFTER you ordered 🤬 not worth the bs & hassle. Stick w/sold & shipped by Amazon & you'll get it when you're supposed too! 👍😁"
                Topics / Keyphrases: ['Third party seller issues on Amazon', 'Inconsistent delivery from Amazon', 'Higher prices from third party sellers', 'Amazon reliability for product delivery', 'Comfortable work boots with steel toe caps', 'Value of quality footwear', 'Frustration with Amazon gift cards', 'Persistent error in Amazon App', 'User frustration with Amazon services', 'Considering parting ways with Amazon']
                Label: 4

                20. Comment: "convenient if you can wait for shipping time ... they usually have what I'm look for ... at a decent price. Sometimes parts/items get grouped in with other products under different description/name. So when you are searching for a specific item and you type in the item it comes up with something different or says not found ... but to only discover it by chance while searching . I like Amazon for the most part and suggest it to others."
                Topics / Keyphrases: ['user experience', 'Amazon Prime Member', 'gift card redemption', 'item search', 'Amazon inventory', 'impulse purchases', 'Amazon App features', 'wish list management', 'product review', 'online shopping behavior']
                Label: 4

                21. Comment: "I wouldn't have survived without Amazon. I am so grateful to them for their caring, first class service"
                Topics / Keyphrases: ['Amazon app usage for retired user', 'Checking global trends on Amazon', 'Potential shortage awareness', 'Value for gamers on Amazon', 'Improving foresight in shopping', 'Satisfying user experience', 'Timely delivery from Amazon', 'Competitive and affordable pricing', 'Convenient online shopping', 'Excellent customer service']
                Label: 5

                22. Comment: "IT'S WONDERFUL SUPER MARKET IN ALL OF THE WORLD"
                Topics / Keyphrases: ['user experience', 'Amazon App', 'Google Play Store', 'user comments', 'positive review', 'purchasing products', 'app evaluation', 'e-commerce', 'finding products', 'initial stage']
                Label: 5

                23. Comment: "Amazon made it possible for me to finally fulfill my lifelong dream of becoming a hermit. good bye crowd, I'm digging it, and bugging out. thank you Amazon"
                Topics / Keyphrases: ['Amazon app user experience', 'lifelong dream Ms. Wendy d. Woods', 'new to e-commerce app', 'front door signatures delivery', 'digging it positive experience', 'user satisfaction thank you Amazon', 'waiting for package delivery', 'Google Play Store download', 'hermit lifestyle goodbye crowd', 'thankful for service bugging out']
                Label: 5

                24. Comment: "Amazon is my 1 stop shopping site!! They are an awesome organization that has always taken care of ALL my shopping and customer service needs!!!"
                Topics / Keyphrases: ['Amazon shopping', 'user experience', 'product variety', 'online shopping', 'customer satisfaction', 'Amazon app', 'Google Play Store', 'user comments', 'app functionality', 'e-commerce']
                Label: 5

                25. Comment: "I wouldn't have survived without Amazon. I am so grateful to them for their caring, first class service"
                Topics / Keyphrases: ['Amazon app usage for retired user', 'Checking global trends on Amazon', 'Potential shortage awareness', 'Value for gamers on Amazon', 'Improving foresight in shopping', 'Satisfying user experience', 'Timely delivery from Amazon', 'Competitive and affordable pricing', 'Convenient online shopping', 'Excellent customer service']
                Label: 5


                Please provide the sentiment classification as a single number (1, 2, 3, 4, or 5) and nothing more for the following comment:

                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        # model="llama3-8b-8192",
        model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [ ]:
len(list)

In [ ]:
rest = train_data[len(list):]
for index, row in rest.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - very negative, 2 - negative, 3 - neutral, 4 - positive, 5 - very positive. Here are examples to guide your classification:

                1. Comment: "Not sure why Amazon thinks it's OK to hijack my long press menu to add their stupid search icon there with no option to remove it. Oh well, now I have one less app taking up space on my phone."
                Topics / Keyphrases: ['social media', 'digital marketing', 'email marketing', 'content marketing', 'social media marketing', 'search engine optimization', 'influencer marketing', 'mobile marketing', 'video marketing', 'email campaigns']
                Label: 1

                2. Comment: "App is now incredibly laggy to the point I'm unable to scroll down to read reviews or look at pictures. Makes it impossible to shop. 0/5 stars. Can't even use the app."
                Topics / Keyphrases: ['Amazon App problems', 'User experience decline', 'App stability issues', 'Recent update difficulties', '5 seconds load time', 'Negative feedback', 'Functionality errors', 'Requires immediate fix', 'User frustration', 'Potential loss of customer']
                Label: 1

                3. Comment: "The absurd experience I've had with Amazon over these past 2 weeks is nothing short of horrendous. I'm all for employing foreign people to deal with customer service but if they themselves are unable to properly handle serious issues what is the point. I have had to spend hours in my bank getting my card changed all so I can make my purchases and your agent has me remove and re-add it only for it to not be accepted then they transfer me 5 times and ask me the same stupid questions"
                Topics / Keyphrases: ['Amazon delivery issues', 'long delivery times', 'user frustrations', 'Prime membership concerns', 'app tracking problems', 'order checking difficulties', 'medical supply delivery', 'customer service communication', 'electric bicycle payment issues', 'inaccurate orders and refunds']
                Label: 1

                4. Comment: "terrible customer service, and they WILL send you broken and or defective products, or nothing at all! prime sucks, they treat you like trash weather you are a prime member who has spent thousands of dollars, or someone whos never spent a dime. seriously, go to the store yourself. amazon corps should be dismantled."
                Topics / Keyphrases: ['higher prices on Amazon', 'product display issues on Amazon', 'frustration with Amazon shopping experience', 'time-consuming product sorting', 'sponsored products concerns on Amazon', 'fake products on Amazon', 'need for seller verification', 'customer dissatisfaction with product display', 'poor delivery experience', 'terrible customer service']
                Label: 1

                5. Comment: "Cannot disable product videos. You search for something and right away videos start playing in the product pictures. Why you can't disable this. I want to focus on shipping not on idiotic videos! This feature is damn, unethical and borderline illegal. You need to be ashamed pushing this to your customers."
                Topics / Keyphrases: ['User experience issue with Amazon App', 'Inclusion of product videos in product pictures', 'Limitations in disabling product videos', 'Customer complaints on Google Play Store', 'Preference for product images over video demonstrations', 'Professional users' difficulty with video feature', 'Annoyance with TikTok-style influencers', 'Product touching element in videos', 'Ethical concerns over unrequested videos', 'Search results affected by unwanted videos']
                Label: 1

                6. Comment: "Too slow, i honestly don't know what happened after the new update it got too laggy, the app started crashing and it became a pain to shop from the app"
                Topics / Keyphrases: ['Amazon App issues', 'user experience decline', 'functionality affected', 'slow performance', 'app crashing', 'order tracking problem', 'money transfer delay', 'constant crashes', 'specific date bug', 'security concern']
                Label: 2

                7. Comment: "Latest app version is unreasonably buggy, especially with friend wish lists. They work fine directly from the website, but why have an app if we're constantly being forced back to the website due to flakes in the app. Google's inability (intent?) to play well with Kindle does make me wonder about the rest of the Amazon."
                Topics / Keyphrases: ['Amazon App performance issues', 'slow loading on multiple phones', 'hard to navigate UI', 'Google-Kindle conflict', 'forced back to website', 'Amazon reliability', 'minimal investment in app', 'connecting issues with Blizzard account', 'increasing prices and reducing service', 'user frustration']
                Label: 2

                8. Comment: "would love to Bulk my order to make it easier for Amazon, but when they can't get simple promised deliveries done, why should I try to help. if they tell you next day delivery it's going to take 2 or 3 days. like it or not. I don't care for that new way of over promising."
                Topics / Keyphrases: ['Amazon delivery issues', 'promised vs actual delivery times', 'user experience and Prime membership', 'prices increase with items', 'monopoly concerns and delayed deliveries', 'payment processing problems', 'damaged and stolen packages', 'product quality and refund processing', 'cost comparison with other vendors', 'alternatives to Amazon']
                Label: 2

                9. Comment: "The app for Android is incredibly inferior to the Apple app. I haven't been able to open my recent orders for months, it spins and then I get an error. I'm not sure why that functionality hasn't been fixed yet."
                Topics / Keyphrases: ['Amazon App issues', 'user experience decline', 'functionality affected', 'slow performance', 'app crashing', 'order tracking problem', 'money transfer delay', 'constant crashes', 'specific date bug', 'security concern']
                Label: 2

                10. Comment: "I'm tired of major app changes without warning. Perhaps user input should count for something. Can't stand having the search bar at the bottom now. Considering uninstalling and just using the website."
                Topics / Keyphrases: ['Amazon App changes', 'user experience', 'feedback on changes', 'Google Play Store comments', 'app design', 'lack of warning', 'user input', 'search bar location', 'app performance issues', 'user complaints']
                Label: 2

                11. Comment: "Please give an option to choose the new or the classic wish list interface. The new interface is completely unhelpful and only adds more clicks/taps to get to where I really want to be. Not to mention, for privacy reasons maybe one doesn't want *everything* appearing on the landing page."
                Topics / Keyphrases: ['Amazon App search bar location', 'User interface preference', 'Application features', 'User feedback', 'Instructional videos', 'Product links', 'Design change', 'User frustration', 'Wish list design', 'End user control']
                Label: 3

                12. Comment: "its okay. can you please bring back the ability to remove suggestions im not interested in. please allow me to either delete older purchases or make sure i stop being advertised stuff i absolutely do not want"
                Topics / Keyphrases: ['social media', 'digital marketing', 'email marketing', 'content marketing', 'social media marketing', 'search engine optimization', 'influencer marketing', 'mobile marketing', 'video marketing', 'email campaigns']
                Label: 3

                13. Comment: "It could be better but it is good. I think when you want to move some items from one folder to another I wish you could select more than one item at a time to move at once."
                Topics / Keyphrases: ['Amazon App user experience', 'app speed and performance', 'product prices and cost', 'shopping frequency and orders', 'delivery time and issues', 'specific delivery location', 'app interface and organization', 'shipping cost', 'dark mode and night time', 'multiple items selection and folders']
                Label: 3

                14. Comment: "Really like using this app as a disabled person it helps me get things that I can't get out to get. Lately though it's been crashing every time I go on, hopefully this is resolved soon."
                Topics / Keyphrases: ['Amazon App issues', 'Digital music errors', 'User frustrations', 'Inability to buy Kindle books', 'Lag when scrolling', 'Positive opinion vs. purchase issues', 'Problems with favorites list', 'Delivery experience issues', 'Searching for items challenges', 'App usability for disabled persons']
                Label: 3

                15. Comment: "The app is good and does the job. There is though no control on the fake reviews for products. The app also does not control any fake products. Unfortunately buyers trust the app and end up buying fake products."
                Topics / Keyphrases: ['Amazon App notifications', 'user preference', 'spam notifications', 'unwanted ads', 'package delivery notifications', 'user complaints', 'poorly rated products', 'app flaws', 'user interface update', 'buyer protection']
                Label: 3

                16. Comment: "So much choice. Great prices. Sometimes returns options are limited, but most sellers and Amazon are generally great at offering flexibility. Be wary of the default being to add a delivery charge for fast delivery and never ending offers of Prime!"
                Topics / Keyphrases: ['Amazon app purchases experience', 'Item quality and price concerns', 'Value for money decreasing', 'Smooth return process', 'Delivery expectations vs reality', 'Price discrepancy in app', 'Prime subscription value', 'Limited returns flexibility', 'International shipping cost', 'E-commerce packaging issues']
                Label: 4

                17. Comment: "Great place for wide range of products at the best prices. my only issue is the amount of time it takes to receive a refund. if not for the wait time would have been 5 starts. customer service not great but better than a lot of others that I have come across."
                Topics / Keyphrases: ['Amazon App experience', 'Higher prices and corporate profits', 'Employee benefits and pay', 'Healthcare and user preferences', 'Late deliveries and delay in transit', 'Warehouse processing and user satisfaction', 'Product delivery and return process', 'Checkout process and cost calculation', 'Wide range of products and Prime membership', 'Customer service and order discrepancies']
                Label: 4

                18. Comment: "Easy app, good product and pretty fast delivery for the most part."
                Topics / Keyphrases: ['Amazon App', 'user experience', 'product availability', 'positive review', 'online shopping', 'customer satisfaction', 'in-app customer reviews', 'rating accuracy', 'fast delivery', 'customer service']
                Label: 4

                19. Comment: "When everywhere else fails get it on Amazon 😂 Don't care for those 3rd party sellers when they tell you it was delivered & was not or we have to fight to get our products from them, they charge a higher price AFTER you ordered 🤬 not worth the bs & hassle. Stick w/sold & shipped by Amazon & you'll get it when you're supposed too! 👍😁"
                Topics / Keyphrases: ['Third party seller issues on Amazon', 'Inconsistent delivery from Amazon', 'Higher prices from third party sellers', 'Amazon reliability for product delivery', 'Comfortable work boots with steel toe caps', 'Value of quality footwear', 'Frustration with Amazon gift cards', 'Persistent error in Amazon App', 'User frustration with Amazon services', 'Considering parting ways with Amazon']
                Label: 4

                20. Comment: "convenient if you can wait for shipping time ... they usually have what I'm look for ... at a decent price. Sometimes parts/items get grouped in with other products under different description/name. So when you are searching for a specific item and you type in the item it comes up with something different or says not found ... but to only discover it by chance while searching . I like Amazon for the most part and suggest it to others."
                Topics / Keyphrases: ['user experience', 'Amazon Prime Member', 'gift card redemption', 'item search', 'Amazon inventory', 'impulse purchases', 'Amazon App features', 'wish list management', 'product review', 'online shopping behavior']
                Label: 4

                21. Comment: "I wouldn't have survived without Amazon. I am so grateful to them for their caring, first class service"
                Topics / Keyphrases: ['Amazon app usage for retired user', 'Checking global trends on Amazon', 'Potential shortage awareness', 'Value for gamers on Amazon', 'Improving foresight in shopping', 'Satisfying user experience', 'Timely delivery from Amazon', 'Competitive and affordable pricing', 'Convenient online shopping', 'Excellent customer service']
                Label: 5

                22. Comment: "IT'S WONDERFUL SUPER MARKET IN ALL OF THE WORLD"
                Topics / Keyphrases: ['user experience', 'Amazon App', 'Google Play Store', 'user comments', 'positive review', 'purchasing products', 'app evaluation', 'e-commerce', 'finding products', 'initial stage']
                Label: 5

                23. Comment: "Amazon made it possible for me to finally fulfill my lifelong dream of becoming a hermit. good bye crowd, I'm digging it, and bugging out. thank you Amazon"
                Topics / Keyphrases: ['Amazon app user experience', 'lifelong dream Ms. Wendy d. Woods', 'new to e-commerce app', 'front door signatures delivery', 'digging it positive experience', 'user satisfaction thank you Amazon', 'waiting for package delivery', 'Google Play Store download', 'hermit lifestyle goodbye crowd', 'thankful for service bugging out']
                Label: 5

                24. Comment: "Amazon is my 1 stop shopping site!! They are an awesome organization that has always taken care of ALL my shopping and customer service needs!!!"
                Topics / Keyphrases: ['Amazon shopping', 'user experience', 'product variety', 'online shopping', 'customer satisfaction', 'Amazon app', 'Google Play Store', 'user comments', 'app functionality', 'e-commerce']
                Label: 5

                25. Comment: "I wouldn't have survived without Amazon. I am so grateful to them for their caring, first class service"
                Topics / Keyphrases: ['Amazon app usage for retired user', 'Checking global trends on Amazon', 'Potential shortage awareness', 'Value for gamers on Amazon', 'Improving foresight in shopping', 'Satisfying user experience', 'Timely delivery from Amazon', 'Competitive and affordable pricing', 'Convenient online shopping', 'Excellent customer service']
                Label: 5


                Please provide the sentiment classification as a single number (1, 2, 3, 4, or 5) and nothing more for the following comment:

                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        # model="llama3-8b-8192",
        model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [ ]:
len(list)

In [ ]:
# list.pop()

In [ ]:
list

In [ ]:
train_data['predictedSentiment'] = pd.Series(list)

In [ ]:
train_data

In [ ]:
import re

def contains_number(s):
    return any(char.isdigit() for char in s)

def findNumbers(string):
    # Find all numbers in the string
    numbers = None
    if(contains_number(string)):
        numbers = re.findall(r'\d+', string)
    else:
        numbers = ""
    
    # Convert the numbers to integers
    
    numbers = [int(x) for x in numbers]
    
    if(len(numbers)>0):
        x = numbers[0]-1
    else:
        x = -1
    return x

In [ ]:
formated = train_data['predictedSentiment'].apply(findNumbers)

In [ ]:
train_data['predictedSentiment'] = formated

In [ ]:
train_data

In [ ]:
train_data.to_csv('mixtralOutput/topics/filenameAmazonFewShotMixtral-T.csv', index=False)

In [ ]:
results = pd.read_csv("mixtralOutput/topics/filenameAmazonFewShotMixtral-T.csv")

In [ ]:
results

In [ ]:
results['predictedSentiment'].value_counts().sort_index()

In [ ]:
filtered_df = results[results["predictedSentiment"] == -1]

In [ ]:
filtered_df

In [ ]:
newResults = results[results["predictedSentiment"] != -1]

In [ ]:
newResults

In [ ]:
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def cm_analysis(y_true, y_pred, filename, labels, classes, ymap=None, figsize=(17,17)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      classes:   aliases for the labels. String array to be shown in the cm plot.
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    sns.set(font_scale=2.8)

    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.2f%%\n%d/%d' % (p, c, s)
            #elif c == 0:
            #    annot[i, j] = ''
            else:
                annot[i, j] = '%.2f%%\n%d' % (p, c)
    cm = confusion_matrix(y_true, y_pred, labels=labels, normalize='true')
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm = cm * 100
    cm.index.name = 'True Label'
    cm.columns.name = 'Predicted Label'
    fig, ax = plt.subplots(figsize=figsize)
    plt.yticks(va='center')

    sns.heatmap(cm, annot=annot, fmt='', ax=ax, xticklabels=classes, cbar=True, cbar_kws={'format':PercentFormatter()}, yticklabels=classes, cmap="Blues")
    plt.savefig(f'{filename}.png',  bbox_inches='tight')
    plt.savefig(f'{filename}.svg',  bbox_inches='tight')

In [ ]:
# Generate the confusion matrix
unique_labels = np.sort(newResults['scoreSA'].unique())
mapping = {0: 'Very Negative (1)', 1: 'Negative (2)', 2: 'Neutral (3)', 3: 'Positive (4)', 4: 'Very Positive (5)'}

# Applying mapping
classes = [mapping[value] for value in unique_labels]

In [ ]:
cm_analysis(newResults['scoreSA'], newResults['predictedSentiment'], 'API', unique_labels, classes)

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support, mean_absolute_error

accuracy = accuracy_score(newResults['scoreSA'], newResults['predictedSentiment'])
precision_ALL = precision_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
recall_ALL = recall_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
f1_ALL = f1_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
mae_ALL = mean_absolute_error(newResults['scoreSA'], newResults['predictedSentiment'])
precision, recall, f1, support = precision_recall_fscore_support(newResults['scoreSA'], newResults['predictedSentiment'])

In [ ]:
# Calculate accuracy for each class
for i in range(len(unique_labels)):
    temp = newResults[newResults["scoreSA"] == i]
    value = accuracy_score(temp['scoreSA'], temp['predictedSentiment'])
    print(f'Label:\t{i}\t-\tValue:\t{value}')

In [ ]:
accuracy

In [ ]:
balanced_accuracy = balanced_accuracy_score(results['scoreSA'], results['predictedSentiment'])

In [ ]:
balanced_accuracy

In [ ]:
precision_ALL

In [ ]:
f1_ALL

In [ ]:
recall_ALL

In [ ]:
mae_ALL

In [ ]:
f1

In [ ]:
precision

In [ ]:
recall

In [ ]:
support